# Генерация кратких резюме текстов моделью mT5 дообученной на основе датасета новостей с сайта Lenta.ru

## Запускаем установку необходимых библиотек

In [ ]:
! pip install transformers transformers[torch] datasets sentencepiece
! pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 55.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 54.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 78.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 29.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 84.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 65.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 21.4 MB/s eta 0:00:00


## Чтобы библиотеки заработали, обязательно перезапускаем среду выполнения
После этого строки выше повторно не запускаем, выполняем код ниже.

# Резюмирование текстов

## Настройки

In [ ]:
MAX_NEW_TOKENS=400
MODEL_DIR='i-k-a/ru_summarization_lenta_model_mt5-base_7_epochs_1024'

## Тест генерации резюме по выборке текстов

### Загрузка датасета Lenta

Загрузим и подготовим датасет из новостных статей и резюме с сайта Lenta.ru

In [ ]:
import pandas as pd
# df = pd.read_csv('lenta.csv', index_col=None)
df = pd.read_csv('http://fotolx.ru/temp/DS/new_lenta.csv', index_col=None)

In [ ]:
from datasets import Dataset
dataset = Dataset.from_dict({'Description':df['Description'],'Text':df['Text']})

### Создаем токенизатор и модель

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained(MODEL_DIR)
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_DIR)

(…)pochs/resolve/main/tokenizer_config.json:   0%|          | 0.00/827 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

(…)chs/resolve/main/special_tokens_map.json:   0%|          | 0.00/416 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:473: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


(…)5-base_4_epochs/resolve/main/config.json:   0%|          | 0.00/827 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.33G [00:00<?, ?B/s]

### Генерируем резюме по выборке текстов

In [ ]:
test_data = pd.DataFrame(dataset).sample(15)
for i, row in test_data.iterrows():
  text = row['Text']
  description = row['Description']
  inputs = tokenizer(text, return_tensors="pt").input_ids
  outputs = model.generate(inputs, max_new_tokens=MAX_NEW_TOKENS, do_sample=False)
  result = tokenizer.decode(outputs[0], skip_special_tokens=True)
  # result = result[:result.rfind('.')]
  print(f'Оригинальное резюме: "{description}"\n\nРезюме от нейросети: "{result}"\n\nИсходный текст: "{text}"')
  print('----------------------------------------------------')
  pass

Оригинальное резюме: "
    В Екатеринбурге суд арестовал 16-летнего подростка за расправу над 14-летней подругой, ее матерью и псом породы шпиц. Подозреваемый взят под стражу и отправлен в СИЗО до 13 января 2024 года. В ходе заседания стало известно, что подросток ранее состоял на учете в службе по делам несовершеннолетних за грабеж.
  "

Резюме от нейросети: "В Екатеринбурге суд арестовал 16-летнего подростка за расправу над 14-летней подругой, ее матерью и псом породы шпиц. Подозреваемый взят под стражу и отправлен в СИЗО до 13 января 2024 года. В ходе заседания стало известно, что подросток ранее состоял на учете в службе по делам несовершеннолетних за грабеж, совершенный в 2022 году. Он тогда избежал наказания из-за возраста."

Исходный текст: "В  Екатеринбурге  суд арестовал 16-летнего подростка за расправу над 14-летней подругой, ее матерью и псом породы шпиц. Об этом сообщает  E1.ru . Подозреваемый взят под стражу и отправлен в СИЗО до 13 января 2024 года. В ходе заседания стало

## Генерация единичного резюме текста

Необходимо предварительно выполнить ячейку "Настройки".

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
tokenizer = AutoTokenizer.from_pretrained(MODEL_DIR)
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_DIR)
text = input('Введите текст:')
inputs = tokenizer(text, return_tensors="pt").input_ids
outputs = model.generate(inputs, max_new_tokens=MAX_NEW_TOKENS, do_sample=False)
result = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(f'Резюме от нейросети: "{result}"\n\nИсходный текст: "{text}"')

Введите текст:Сильная магнитная буря вызвала редкое полярное сияние на юге России. Красное свечение могли увидеть жители Крыма, Ростовской и Воронежской областей, а также новых регионов страны, передает РБК.  Астроном Александр Иванов в беседе с «Комсомольской Правдой» объяснил, что это явление является очень редким для южных регионов. Полярное сияние длится от пяти минут до получаса и происходит однократно.  «Полярное сияние возникает от протонных ударов по магнитосфере. Магнитные бури возникли недавно, с обеда 5 ноября. Продолжаются три часа подряд. Свечение как раз связано с этим. Фактически северное сияние, механизм такой же», — отметил он.  В конце сентября в России засняли редкое по красоте полярное сияние. В ночь на 25 сентября жители Советска могли увидеть в небе над рекой Неман полярное сияние в ярком зеленом и алом спектре.
Резюме от нейросети: "В силу магнитная буря вызвала редкое полярное сияние на юге России. Красное свечение могли увидеть жители Крыма, Ростовской и Вороне